# URL 檢查器 - Google Colab 版本

**功能：** 批量檢測 URL 的有效性，支持多種檔案格式上傳、詳細報告生成和結果導出

**版本：** 2.0.0

**環境：** Google Colab (Python 3.9+)

---

## 快速說明

本筆記本提供完整的 URL 驗證工作流程：

1. ✅ **自動安裝依賴**：一鍵安裝所需套件
2. ✅ **克隆倉庫**：從 GitHub 導入最新代碼
3. ✅ **檔案上傳**：支持 TXT / CSV / JSON / XML 格式
4. ✅ **批量檢測**：多線程並發，自動調整最優線程數
5. ✅ **報告顯示**：Colab 中直接查看檢測結果表格
6. ✅ **結果導出**：CSV 或 Excel 格式

---

## 使用步驟

按順序執行下方 Cell 即可：
1. **Cell 1**：環境準備（安裝依賴 + 克隆倉庫）
2. **Cell 2**：初始化和上傳檔案
3. **Cell 3**：解析檔案並查看提取的 URL
4. **Cell 4**：批量檢測 URL
5. **Cell 5**：顯示詳細報告
6. **Cell 6**：導出結果（可選）
7. **Cell 7**：統計分析和快速查詢

---

## [Cell 1] 環境準備：安裝依賴 & 克隆倉庫

執行此 Cell 自動安裝所有必需套件並克隆 GitHub 倉庫。

**預計耗時：** 30-45 秒

In [ ]:
#!/usr/bin/env python3
# @title 🚀 環境準備：一鍵安裝

import subprocess
import sys
import os
from datetime import datetime

print("="*80)
print("URL 檢查器 - Google Colab 環境準備")
print("="*80)
print(f"\n開始時間: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

# 步驟 1：安裝依賴
print("[1/3] 正在安裝依賴套件...")
print("-"*80)
try:
    subprocess.check_call([
        sys.executable, "-m", "pip", "install", "-q",
        "requests>=2.28.0",
        "pandas>=1.3.0",
        "psutil>=5.9.0",
        "openpyxl>=3.8.0"
    ])
    print("✓ 依賴安裝完成\n")
except Exception as e:
    print(f"✗ 依賴安裝失敗: {str(e)}\n")
    sys.exit(1)

# 步驟 2：克隆倉庫
print("[2/3] 正在克隆 GitHub 倉庫...")
print("-"*80)
try:
    if not os.path.exists("url-validator-colab"):
        subprocess.run(["git", "clone", 
                       "https://github.com/hoyaani/url-validator-colab.git"],
                      check=True, capture_output=True)
        print("✓ 倉庫克隆完成")
    else:
        print("✓ 倉庫已存在")
except Exception as e:
    print(f"✗ 倉庫克隆失敗: {str(e)}")
    sys.exit(1)

# 步驟 3：切換工作目錄
print("\n[3/3] 配置工作環境...")
print("-"*80)
try:
    os.chdir("url-validator-colab")
    print(f"✓ 工作目錄: {os.getcwd()}")
except Exception as e:
    print(f"✗ 環境配置失敗: {str(e)}")
    sys.exit(1)

# 驗證安裝
print("\n" + "="*80)
print("環境驗證")
print("="*80)
try:
    import requests, pandas, psutil, openpyxl
    print(f"✓ requests: {requests.__version__}")
    print(f"✓ pandas: {pandas.__version__}")
    print(f"✓ psutil: {psutil.__version__}")
    print(f"✓ openpyxl: {openpyxl.__version__}")
    print(f"\n✓ 所有依賴安裝成功！")
except ImportError as e:
    print(f"✗ 依賴驗證失敗: {str(e)}")
    sys.exit(1)

print(f"\n完成時間: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("\n" + "="*80)
print("✅ 環境準備完成，請執行 Cell 2")
print("="*80)

## [Cell 2] 初始化檢查器 & 上傳檔案

初始化 URL 檢查器，系統將自動偵測資源並建議最優配置。

隨後彈出檔案選擇框，上傳包含 URL 的檔案（支持 TXT / CSV / JSON / XML）。

In [ ]:
#!/usr/bin/env python3
# @title 📁 初始化檢查器 & 上傳檔案

import sys
import os

# 導入主模塊
sys.path.insert(0, os.getcwd())
from main import URLValidatorColab
import pandas as pd

print("="*80)
print("初始化 URL 檢查器")
print("="*80)
print()

# 初始化檢查器
validator = URLValidatorColab()

print()
print("="*80)
print("檔案上傳")
print("="*80)
print()
print("支持格式：")
print("  • TXT：每行一個 URL（推薦）")
print("  • CSV：自動掃描所有列尋找 URL")
print("  • JSON：遞迴掃描嵌套結構")
print("  • XML：掃描元素文本和屬性")
print()
print("提示：點擊下方『選擇檔案』按鈕上傳本地檔案")
print()

# 上傳檔案
filepath = validator.upload_file_from_colab()

if filepath:
    print(f"✓ 檔案已上傳: {os.path.basename(filepath)}")
    # 保存 validator 和 filepath 到全局變量供後續 Cell 使用
    globals()['validator'] = validator
    globals()['filepath'] = filepath
else:
    print("✗ 未選擇檔案，程序中止")
    sys.exit(1)

print()
print("="*80)
print("✅ 初始化完成，請執行 Cell 3")
print("="*80)

## [Cell 3] 解析檔案 & 提取 URL

自動解析上傳的檔案，提取所有 URL 並去重。

顯示提取的 URL 列表和基本統計信息。

In [ ]:
#!/usr/bin/env python3
# @title 🔍 解析檔案 & 提取 URL

import os

if 'validator' not in globals() or 'filepath' not in globals():
    print("✗ 錯誤：請先執行 Cell 2")
    exit()

print("="*80)
print("檔案解析")
print("="*80)
print()

# 解析檔案
urls = validator.parse_url_file(filepath)

if not urls:
    print("✗ 未成功提取 URL")
    exit()

print()
print("="*80)
print("提取結果統計")
print("="*80)
print(f"\n✓ 成功提取 {len(urls)} 個 URL（已去重）")
print(f"  檔案名: {os.path.basename(filepath)}")
print(f"  檔案大小: {os.path.getsize(filepath) / 1024:.2f} KB")

# 顯示前 10 個 URL
print(f"\n前 {min(10, len(urls))} 個 URL：")
print("-"*80)
for i, url in enumerate(urls[:10], 1):
    print(f"{i:2d}. {url}")

if len(urls) > 10:
    print(f"... 以及其他 {len(urls) - 10} 個 URL")

# 分析 URL 協議
from urllib.parse import urlparse
protocols = {}
for url in urls:
    try:
        parsed = urlparse(url)
        scheme = parsed.scheme or "無協議"
        protocols[scheme] = protocols.get(scheme, 0) + 1
    except:
        pass

print("\n\n協議分佈：")
print("-"*80)
for scheme, count in sorted(protocols.items(), key=lambda x: x[1], reverse=True):
    percentage = (count / len(urls)) * 100
    bar = "█" * int(percentage / 5)
    print(f"{scheme:12s}: {count:3d} 個 ({percentage:5.1f}%) {bar}")

# 保存 URL 列表到全局變量
globals()['urls'] = urls

print()
print("="*80)
print("✅ URL 提取完成，請執行 Cell 4")
print("="*80)

## [Cell 4] 批量檢測 URL

使用多線程並發檢測所有 URL 的連通性（DNS 解析 + HTTP 請求）。

實時顯示檢測進度。

In [ ]:
#!/usr/bin/env python3
# @title ⚙️ 批量檢測 URL

if 'validator' not in globals() or 'urls' not in globals():
    print("✗ 錯誤：請先執行 Cell 2-3")
    exit()

print("="*80)
print("URL 連通性檢測")
print("="*80)
print()
print(f"檢測配置：")
print(f"  • DNS 超時: {validator.config['dns_timeout']} 秒")
print(f"  • HTTP 超時: {validator.config['http_timeout']} 秒")
print(f"  • 並發線程: {validator.config['max_workers']} 個")
print()

# 執行檢測
results_df = validator.validate_urls_batch(urls)

if results_df.empty:
    print("✗ 檢測失敗")
    exit()

# 保存結果到全局變量
globals()['results_df'] = results_df

print()
print("="*80)
print("✅ 檢測完成，請執行 Cell 5 查看報告")
print("="*80)

## [Cell 5] 顯示詳細報告

在 Colab 中直接顯示檢測結果表格，包含統計摘要和完整的 URL 驗證信息。

In [ ]:
#!/usr/bin/env python3
# @title 📊 顯示詳細報告

if 'validator' not in globals() or 'results_df' not in globals():
    print("✗ 錯誤：請先執行 Cell 4")
    exit()

# 顯示報告
validator.display_report(results_df)

# 額外統計
print("\n\n" + "="*100)
print("失敗原因統計")
print("="*100)

failed_df = results_df[results_df['status'] == '失敗']
if len(failed_df) > 0:
    reason_counts = failed_df['reason'].value_counts()
    for reason, count in reason_counts.items():
        percentage = (count / len(failed_df)) * 100
        print(f"  • {reason}: {count} 個 ({percentage:.1f}%)")
else:
    print("  • 所有 URL 檢測成功！🎉")

## [Cell 6] 導出結果

將檢測報告導出為 CSV 或 Excel 格式，並自動下載到本地。

In [ ]:
#!/usr/bin/env python3
# @title 💾 導出結果

import os

if 'validator' not in globals() or 'results_df' not in globals():
    print("✗ 錯誤：請先執行 Cell 4-5")
    exit()

print("="*80)
print("導出選項")
print("="*80)
print()
print("請選擇導出格式：")
print()
print("1. CSV 格式 - 輕量級、通用格式（推薦快速分享）")
print("2. Excel 格式 - 功能豐富、表格美觀（推薦專業報告）")
print("0. 跳過導出")
print()

# 取得用戶選擇
choice = input("請輸入選擇 (0-2): ").strip()

print()
print("="*80)
print("導出進行中")
print("="*80)
print()

try:
    from google.colab import files
    
    if choice == "1":
        print("[1/2] 生成 CSV 報告...")
        csv_path = validator.export_report(results_df, "csv")
        if csv_path:
            print(f"[2/2] 下載檔案...")
            files.download(csv_path)
            print(f"\n✓ 導出完成")
    elif choice == "2":
        print("[1/2] 生成 Excel 報告...")
        xlsx_path = validator.export_report(results_df, "excel")
        if xlsx_path:
            print(f"[2/2] 下載檔案...")
            files.download(xlsx_path)
            print(f"\n✓ 導出完成")
    elif choice == "0":
        print("✓ 跳過導出")
    else:
        print("✗ 無效選擇")
except Exception as e:
    print(f"✗ 導出失敗: {str(e)}")

print()
print("="*80)

## [Cell 7] 統計分析 & 快速查詢

對檢測結果進行深度分析，按狀態、域名、協議等多維度統計。

In [ ]:
#!/usr/bin/env python3
# @title 📈 統計分析 & 快速查詢

from urllib.parse import urlparse
import pandas as pd

if 'results_df' not in globals():
    print("✗ 錯誤：請先執行 Cell 4-5")
    exit()

print("="*100)
print("深度統計分析")
print("="*100)
print()

# 1. 按狀態統計
print("【統計 1】按檢測狀態分類")
print("-"*100)
status_counts = results_df['status'].value_counts()
for status, count in status_counts.items():
    percentage = (count / len(results_df)) * 100
    emoji = "✅" if status == "成功" else "❌"
    bar_length = int(percentage / 2)
    bar = "█" * bar_length + "░" * (50 - bar_length)
    print(f"{emoji} {status:4s}: {count:4d} 個 ({percentage:5.1f}%) {bar}")
print()

# 2. 按協議統計
print("【統計 2】按 URL 協議分類")
print("-"*100)
schemes = {}
for url in results_df['url']:
    try:
        scheme = urlparse(url).scheme or "無協議"
        schemes[scheme] = schemes.get(scheme, 0) + 1
    except:
        pass
for scheme, count in sorted(schemes.items(), key=lambda x: x[1], reverse=True):
    percentage = (count / len(results_df)) * 100
    print(f"  • {scheme:12s}: {count:4d} 個 ({percentage:5.1f}%)")
print()

# 3. 按域名統計（Top 10）
print("【統計 3】Top 10 最常檢測的域名")
print("-"*100)
domains = {}
for url in results_df['url']:
    try:
        domain = urlparse(url).netloc
        domains[domain] = domains.get(domain, 0) + 1
    except:
        pass
top_domains = sorted(domains.items(), key=lambda x: x[1], reverse=True)[:10]
for i, (domain, count) in enumerate(top_domains, 1):
    print(f"{i:2d}. {domain:40s}: {count:3d} 次")
print()

# 4. 失敗原因詳細分析
print("【統計 4】失敗原因詳細分析")
print("-"*100)
failed_df = results_df[results_df['status'] == '失敗']
if len(failed_df) > 0:
    reason_groups = failed_df.groupby('reason').size().sort_values(ascending=False)
    for reason, count in reason_groups.items():
        percentage = (count / len(failed_df)) * 100
        print(f"  ✗ {reason}: {count} 個 ({percentage:.1f}%)")
else:
    print("  ✓ 所有 URL 檢測成功，無失敗記錄！")
print()

# 5. 快速查詢
print("="*100)
print("快速查詢")
print("="*100)
print()
print("【查詢 1】查看所有失敗的 URL")
print("-"*100)
if len(failed_df) > 0:
    display(failed_df[['url', 'reason', 'check_time']].reset_index(drop=True))
else:
    print("  無失敗記錄")
print()

print("【查詢 2】查看所有成功的 URL（顯示前 20 個）")
print("-"*100)
success_df = results_df[results_df['status'] == '成功']
if len(success_df) > 0:
    display(success_df[['url', 'reason', 'check_time']].head(20).reset_index(drop=True))
else:
    print("  無成功記錄")
print()

# 保存統計結果
print("="*100)
print("檢測統計摘要")
print("="*100)
print(f"""
總計 URL 數量:      {len(results_df):4d} 個
成功檢測:          {len(success_df):4d} 個 ({len(success_df)/len(results_df)*100:.1f}%)
失敗檢測:          {len(failed_df):4d} 個 ({len(failed_df)/len(results_df)*100:.1f}%)
唯一域名數:        {len(domains):4d} 個
""")

## [Cell 8] 進階用法：自定義篩選與導出

根據不同條件篩選結果，並分別導出特定子集。

In [ ]:
#!/usr/bin/env python3
# @title 🔧 進階用法：自定義篩選與導出

from urllib.parse import urlparse
from datetime import datetime
import os

if 'results_df' not in globals():
    print("✗ 錯誤：請先執行 Cell 4-5")
    exit()

print("="*80)
print("進階用法示例")
print("="*80)
print()

# 示例 1：只提取失敗的 URL
print("【示例 1】提取所有失敗的 URL")
print("-"*80)
failed_urls = results_df[results_df['status'] == '失敗']['url'].tolist()
print(f"失敗 URL 數量：{len(failed_urls)} 個")
if failed_urls:
    for i, url in enumerate(failed_urls[:5], 1):
        print(f"{i}. {url}")
    if len(failed_urls) > 5:
        print(f"... 還有 {len(failed_urls) - 5} 個")
print()

# 示例 2：按 DNS 超時篩選
print("【示例 2】篩選因 DNS 超時而失敗的 URL")
print("-"*80)
dns_timeout_urls = results_df[
    (results_df['status'] == '失敗') & 
    (results_df['reason'].str.contains('DNS|解析超時', na=False))
]
print(f"DNS 超時 URL 數量：{len(dns_timeout_urls)} 個")
if len(dns_timeout_urls) > 0:
    print(dns_timeout_urls[['url', 'reason']].to_string(index=False))
print()

# 示例 3：按 HTTP 狀態碼篩選
print("【示例 3】篩選 HTTP 4xx 錯誤")
print("-"*80)
http_4xx = results_df[
    (results_df['status'] == '失敗') & 
    (results_df['reason'].str.contains('HTTP 4', na=False))
]
print(f"HTTP 4xx 錯誤 URL 數量：{len(http_4xx)} 個")
if len(http_4xx) > 0:
    print(http_4xx[['url', 'reason']].to_string(index=False))
print()

# 示例 4：按域名篩選
print("【示例 4】篩選特定域名的 URL")
print("-"*80)
print("輸入域名關鍵字（如 'google.com'，留空跳過）：")
domain_filter = input("關鍵字: ").strip()
if domain_filter:
    filtered = results_df[results_df['url'].str.contains(domain_filter, na=False)]
    print(f"\n符合'{domain_filter}'的 URL 數量：{len(filtered)} 個")
    if len(filtered) > 0:
        print(filtered[['url', 'status', 'reason']].to_string(index=False))
else:
    print("\n已跳過域名篩選")
print()

# 示例 5：自定義導出
print("【示例 5】自定義導出：僅導出失敗的 URL")
print("-"*80)
try:
    failed_df = results_df[results_df['status'] == '失敗']
    if len(failed_df) > 0:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        export_path = f"/tmp/failed_urls_{timestamp}.csv"
        failed_df[['url', 'reason', 'check_time']].to_csv(
            export_path, 
            index=False, 
            encoding="utf-8-sig"
        )
        print(f"✓ 已導出 {len(failed_df)} 個失敗 URL")
        print(f"  檔案：{os.path.basename(export_path)}")
        
        # 提供下載選項
        from google.colab import files
        print(f"\n下載導出檔案...")
        files.download(export_path)
    else:
        print("✓ 無失敗 URL 可導出")
except Exception as e:
    print(f"✗ 導出失敗：{str(e)}")
print()

print("="*80)
print("✅ 進階用法演示完成")
print("="*80)

## [Cell 9] 自定義配置：調整檢測參數

如果需要更改超時時間、線程數等參數，可以在此 Cell 調整，然後重新執行 Cell 4 檢測。

In [ ]:
#!/usr/bin/env python3
# @title ⚙️ 自定義配置參數

if 'validator' not in globals():
    print("✗ 錯誤：請先執行 Cell 2")
    exit()

print("="*80)
print("自定義配置參數")
print("="*80)
print()

print("當前配置：")
print("-"*80)
for key, value in validator.config.items():
    print(f"  {key:20s}: {value}")
print()

print("可調整參數說明：")
print("-"*80)
print("""
1. dns_timeout (秒)
   • 默認：3 秒
   • 範圍：1-10 秒
   • 說明：DNS 解析等待時間，過短會誤判，過長增加耗時

2. http_timeout (秒)
   • 默認：5 秒
   • 範圍：2-15 秒
   • 說明：HTTP 請求等待時間

3. max_workers (個)
   • 默認：自動計算 (2-10)
   • 範圍：1-15 個
   • 說明：並發線程數，更多線程速度快但占用資源多
""")

print("="*80)
print("常見場景配置建議")
print("="*80)
print("""
【場景 1】網絡不穩定
  dns_timeout: 5
  http_timeout: 8
  max_workers: 2

【場景 2】快速掃描（網絡良好）
  dns_timeout: 2
  http_timeout: 3
  max_workers: 8

【場景 3】目標伺服器敏感（防止被封禁）
  dns_timeout: 3
  http_timeout: 5
  max_workers: 1
""")

print("="*80)
print("修改配置示例")
print("="*80)
print("""
# 修改 DNS 超時時間
validator.config["dns_timeout"] = 5

# 修改 HTTP 超時時間
validator.config["http_timeout"] = 8

# 修改並發線程數
validator.config["max_workers"] = 4

# 一次修改多個參數
validator.config = {
    "dns_timeout": 5,
    "http_timeout": 8,
    "max_workers": 3
}

# 修改後，重新執行 Cell 4 進行檢測
""")

# 提供交互式修改
print("="*80)
print("交互式配置修改")
print("="*80)
print()
print("是否要修改配置? (y/n): ", end="")
choice = input().strip().lower()

if choice == 'y':
    print()
    print("請輸入新的 DNS 超時時間（秒，留空保持不變）:")
    dns_input = input("DNS 超時: ").strip()
    if dns_input:
        try:
            validator.config["dns_timeout"] = int(dns_input)
        except:
            print("✗ 無效輸入")
    
    print()
    print("請輸入新的 HTTP 超時時間（秒，留空保持不變）:")
    http_input = input("HTTP 超時: ").strip()
    if http_input:
        try:
            validator.config["http_timeout"] = int(http_input)
        except:
            print("✗ 無效輸入")
    
    print()
    print("請輸入新的並發線程數（個，留空保持不變）:")
    workers_input = input("並發線程: ").strip()
    if workers_input:
        try:
            validator.config["max_workers"] = int(workers_input)
        except:
            print("✗ 無效輸入")
    
    print()
    print("修改後的配置：")
    print("-"*80)
    for key, value in validator.config.items():
        print(f"  {key:20s}: {value}")
    print()
    print("✓ 配置修改完成，請重新執行 Cell 4 進行檢測")
else:
    print("\n✓ 保持原有配置")

print()
print("="*80)

## 常見問題 (FAQ)

### Q1：為什麼有些 URL 顯示「DNS 解析失敗」？
**A：** 這通常表示域名不存在或 DNS 服務器無法解析。常見原因：
- 域名拼寫錯誤
- 域名已過期或被刪除
- 企業內網域名（無公開 DNS 記錄）
- 網絡連接問題

### Q2：為什麼有些 URL 顯示「HTTP 請求超時」？
**A：** 伺服器響應慢或網絡連接問題。解決方案：
1. 增加超時時間（Cell 9）
2. 檢查網絡連接
3. 檢查伺服器是否在線

### Q3：能否下載檢測報告？
**A：** 可以。執行 Cell 6 後系統會自動彈出下載提示，或使用 Cell 8 進階用法自定義導出。

### Q4：如何處理 Colab 記憶體不足？
**A：** 方案包括：
1. 減少並發線程數（Cell 9）
2. 分批檢測（Cell 8）
3. 重啟 Colab runtime

### Q5：檢測後是否會影響目標伺服器？
**A：** 影響極小：
- 使用 HEAD 方法而非 GET（不下載完整內容）
- 默認線程數為 2-5（避免 DoS）
- 每個 URL 只發送一次請求

**注意：** 大規模掃描可能被視為 DoS 攻擊。若掃描他人網站，應先獲得授權。

### Q6：支援哪些檔案格式？
**A：** 支持 4 種格式：
- **TXT**：每行一個 URL（推薦）
- **CSV**：自動掃描所有列
- **JSON**：遞迴掃描嵌套結構
- **XML**：掃描元素文本和屬性

### Q7：檢測報告能否按特定條件篩選？
**A：** 可以。使用 Cell 8 進階用法，支持按狀態、原因、域名等多維度篩選。

---


## 系統資訊 & 幫助

### 版本資訊

In [ ]:
#!/usr/bin/env python3
# @title 📋 系統資訊 & 版本檢查

import sys
import platform
from datetime import datetime

print("="*80)
print("系統資訊")
print("="*80)
print()

print("Python 環境：")
print(f"  • Python 版本: {sys.version.split()[0]}")
print(f"  • 平台: {platform.platform()}")
print(f"  • 處理器: {platform.processor()}")
print()

print("已安裝套件：")
print("-"*80)
try:
    import requests
    print(f"  ✓ requests: {requests.__version__}")
except:
    print(f"  ✗ requests: 未安裝")

try:
    import pandas
    print(f"  ✓ pandas: {pandas.__version__}")
except:
    print(f"  ✗ pandas: 未安裝")

try:
    import psutil
    print(f"  ✓ psutil: {psutil.__version__}")
except:
    print(f"  ✗ psutil: 未安裝")

try:
    import openpyxl
    print(f"  ✓ openpyxl: {openpyxl.__version__}")
except:
    print(f"  ✗ openpyxl: 未安裝")

print()
print("Colab 環境：")
print("-"*80)
try:
    import google.colab
    print(f"  ✓ Google Colab: 已檢測")
except:
    print(f"  ✗ Google Colab: 未檢測")

print()
print("系統資源：")
print("-"*80)
try:
    import psutil
    cpu_count = psutil.cpu_count(logical=True)
    memory = psutil.virtual_memory()
    print(f"  • CPU 核心數: {cpu_count}")
    print(f"  • 總記憶體: {memory.total / (1024**3):.2f} GB")
    print(f"  • 可用記憶體: {memory.available / (1024**3):.2f} GB")
    print(f"  • 記憶體使用率: {memory.percent:.1f}%")
except:
    print(f"  • 無法取得系統資源訊息")

print()
print("="*80)
print("URL 檢查器資訊")
print("="*80)
print(f"""
版本號: 2.0.0
發佈日期: 2024-05-20
開發語言: Python 3.9+
執行環境: Google Colab
授權協議: MIT License

核心功能:
  ✓ 多格式檔案支持 (TXT/CSV/JSON/XML)
  ✓ DNS 解析檢測
  ✓ HTTP 連通性檢測
  ✓ 多線程並發處理
  ✓ 詳細報告生成
  ✓ 雙格式結果導出 (CSV/Excel)

GitHub: https://github.com/hoyaani/url-validator-colab
""")

print("="*80)
print(f"檢查時間: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)

## 故障排除

### 問題 1：ImportError 找不到模塊
**解決方案：** 請確保已執行 Cell 1 安裝依賴，若仍有問題：
```python
!pip install --upgrade requests pandas psutil openpyxl
```

### 問題 2："找不到倉庫" 錯誤
**解決方案：** 確保 GitHub 連結正確，或手動上傳 `main.py` 檔案到 Colab

### 問題 3：記憶體不足 MemoryError
**解決方案：**
1. 執行 Cell 9 減少線程數
2. 重啟 Colab runtime（菜單 → 重新啟動運行時）
3. 分批次處理 URL

### 問題 4：無法下載報告檔案
**解決方案：** Colab 會話結束後檔案自動刪除，請：
1. 立即下載檔案
2. 或上傳到 Google Drive：
```python
from google.colab import drive
drive.mount('/content/drive')
```

### 問題 5：CSV 編碼亂碼
**解決方案：** 導出時已使用 UTF-8-sig，若仍有問題，使用 Excel 格式

---

## 使用提示 & 最佳實踐

### ✅ 推薦做法

1. **按順序執行 Cell**：先執行 Cell 1-5，再根據需要執行後續 Cell
2. **檢查檔案格式**：確保上傳的檔案格式正確（TXT 最簡單）
3. **定期保存報告**：檢測完成後立即下載報告，避免丟失
4. **合理設置超時時間**：過短會誤判，過長會增加耗時
5. **定期檢查資源**：監控 Colab 記憶體和 CPU 使用率

### ❌ 應避免

1. **同時檢測過多 URL**：分批處理大型列表（超過 1000 個）
2. **過度並發**：線程數過多會導致記憶體不足
3. **短時間內重複檢測同一 URL**：可能被伺服器封禁
4. **未授權掃描他人網站**：需獲得明確授權
5. **忽視超時設置**：可能導致檢測卡頓

---

## 更多資源

### 相關文檔
- [完整 README](https://github.com/hoyaani/url-validator-colab/blob/main/README.md)
- [API 文檔](https://github.com/hoyaani/url-validator-colab#api-文檔)
- [GitHub 倉庫](https://github.com/hoyaani/url-validator-colab)

### 反饋與支持
- 🐛 [報告 Bug](https://github.com/hoyaani/url-validator-colab/issues/new?assignees=&labels=bug&projects=&template=bug_report.md)
- 💡 [功能建議](https://github.com/hoyaani/url-validator-colab/issues/new?assignees=&labels=enhancement&projects=&template=feature_request.md)
- 💬 [討論區](https://github.com/hoyaani/url-validator-colab/discussions)

---

**最後更新：** 2024-05-20  
**版本：** 2.0.0  
**許可證：** MIT License

感謝使用 URL 檢查器！如有問題或建議，歡迎反饋。